# Construction and validation of bioNLP named entity recognition model using deep learning


First install keras-gpu: using


conda create -n deep_nlp_ pip python=3.6 && source activate deep_nlp_


conda install -n deep_nlp_ --yes anaconda tensorflow-gpu && conda install -n deep_nlp_ --yes anaconda keras-gpu 

pip install spacy scikit-learn

Problems: the embeddings I trained is worse than the one from google


Attribution: 
    Some of the code are borrowed from the example code from spacy

In [1]:
import random
import pathlib
import cytoolz
import numpy
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional,CuDNNLSTM
from keras.layers import TimeDistributed
from keras.optimizers import Adam
#import thinc.extra.datasets
from spacy.compat import pickle
import spacy
import pandas as pd
from sklearn import preprocessing
import keras
from sklearn import model_selection
#import seaborn as sns
from sklearn import metrics
#import matplotlib.pyplot as plt

/cellar/users/btsui/anaconda3/envs/deep_nlp_/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#!pip install scikit-learn

In [3]:
#!pip install scikit-learn

### load in SRA data

In [4]:
%%time 
inS_dir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle'

sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.pickle'

srsS=pd.read_pickle(inS_dir)
srsS=pd.Series(data=srsS.values,index=pd.MultiIndex.from_arrays([srsS.index.get_level_values(0),
                                                            srsS.index.get_level_values(1).str.lower()]) )
technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)


CPU times: user 36.5 s, sys: 6.22 s, total: 42.7 s
Wall time: 46.2 s


In [5]:
#srsS.memory_usage()/10**9

In [6]:
#!python -m spacy download en

### load in spacy models

In [7]:
%%time
#nlp=spacy.load('en_vectors_web_lg')
#wikipedia-pubmed-and-PMC-w2v
nlp=spacy.load('en')

CPU times: user 1.72 s, sys: 112 ms, total: 1.84 s
Wall time: 665 ms


### Functions for model compilation and feature extraction

Useful information for understanding the neural network: 

TimeDistributed: https://machinelearningmastery.com/timedistributed-layer-for-long-short-term-memory-networks-in-python/

lr stands for: learning rate

In [8]:


def get_features(docs, max_length):
    docs = list(docs)
    Xs = numpy.zeros((len(docs), max_length), dtype='int32')
    for i, doc in enumerate(docs):
        j = 0
        for token in doc:
            ##rever to word vector
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    return Xs

### subset sra data

In [9]:
#max_length=10
max_sample_per_study_n=100


In [10]:
srsWithText=srsS.index.get_level_values(0).unique()

In [11]:
### take only intersection and than randomize the dataframe
tmpDf=technical_meta_data_df.drop_duplicates('Sample')
tmpInterDf=tmpDf[tmpDf.Sample.isin(srsWithText)]
shuffledDf=tmpInterDf.sample(n=tmpInterDf.shape[0],random_state=0
                                            )
technical_meta_data_df_sub=shuffledDf.groupby('Study').head(n=max_sample_per_study_n)

In [12]:
print ('# of unique studies:',technical_meta_data_df_sub.Study.nunique())

# of unique studies: 106110


### subset data based on entity types 

In [13]:
train_test_ratio=0.8

In [14]:
subsampledSrs=technical_meta_data_df.groupby('Study').head(n=max_sample_per_study_n)['Sample']
#,'dev_stage','phenotype'
myAttribs=['organism','geo_loc_name','strain','sex','tissue','age','genotype','treatment',
           'breed','body_product',
           'disease','cell_line','ethnicity']
#myAttribs=['SCIENTIFIC_NAME','biomaterial_provider','race','population_description','source_name','sex','age','strain','genotype','disease','treatment']
m=srsS.index.get_level_values(0).isin(subsampledSrs.values)
m1=srsS.index.get_level_values(1).isin(myAttribs)
srsS_subS=srsS[m&m1]

In [15]:
myDf=srsS_subS.reset_index()
myDf.columns=['srs','attrib','sentence']

In [16]:
myDf['sentence']=myDf['sentence'].str.replace('_','')

### split data into training and  testing randomly by study levels

It is split by study level to avoid overgeneralization

In [17]:
study_S=technical_meta_data_df_sub['Study'].drop_duplicates()
print (len(study_S))
myNStudies=len(study_S)
train_n=int((myNStudies*train_test_ratio))
train_studies=study_S.sample(n=train_n,random_state=0).values

106110


In [18]:
technical_meta_data_df_sub['Train']=\
    technical_meta_data_df_sub['Study'].isin(train_studies)

/cellar/users/btsui/anaconda3/envs/deep_nlp_/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
train_mask=technical_meta_data_df_sub['Train']
train_samples=technical_meta_data_df_sub['Sample'][train_mask].values
test_samples=technical_meta_data_df_sub['Sample'][~train_mask].values


In [20]:
### set training and testing within the dataframe
#inTrainTestDf=myDf.sample(n=myDf.shape[0]).groupby('attrib').head(n=20000)
cap_size=50000

all_train_df=myDf[myDf.srs.isin(train_samples)]
g=all_train_df.groupby('attrib')
print (g.size().min())
#train_df=g.head(g.size().min())
train_df=all_train_df.sample(n=all_train_df.shape[0]).groupby('attrib').head(n=cap_size)
all_test_df=myDf[myDf.srs.isin(test_samples)]
test_df=all_test_df.sample(n=all_test_df.shape[0]).groupby('attrib').head(cap_size)
#cap_size=g.size().min()




3824


In [21]:
attribVC_train=train_df.attrib.value_counts()#.to_frame()

In [22]:
#attribVC_test.to_frame()

In [23]:
#attribVC_train.to_frame()

### set up LSTM model

use the sklearn encoder going back and forth between classes in string format and integer format

In [24]:
le = preprocessing.LabelEncoder()
le.fit(myAttribs)
nr_classes=len(le.classes_)

In [26]:
nr_hidden=64
max_length=65#, # Shape
dropout=0.5
learn_rate=0.001#, # General NN config
nb_epoch=1#
batch_size=100

In [27]:
lstm_shape={'nr_hidden': 64, 'max_length': max_length, 'nr_class': nr_classes}
lstm_settings={'dropout': 0.5, 'lr': 0.001}

In [28]:
embeddings = nlp.vocab.vectors.data
#embedgigs is vocab.vectors.data


### transform list of freetexts into a matrix of word embeddings

In [29]:
from tqdm import tqdm

In [ ]:
train_texts=train_df.sentence.tolist()
dev_texts=test_df.sentence.tolist()

train_labels=keras.utils.to_categorical(
    le.transform(train_df.attrib.values))
dev_labels=keras.utils.to_categorical(le.transform(test_df.attrib.values))

train_docs = list(tqdm(nlp.pipe(train_texts,n_threads=8),total=len(train_texts)))
dev_docs = list(tqdm(nlp.pipe(dev_texts,n_threads=8),total=len(dev_texts)))

train_X = get_features(train_docs, lstm_shape['max_length'])
dev_X = get_features(dev_docs, lstm_shape['max_length'])


 49%|████▉     | 214001/435265 [03:35<03:43, 991.39it/s]

In [ ]:
##import gc
#gc.collect()

#### Model training


#26 mins per epoch, on a machine with 48 threads, might want to use GPU machine bordeaux for this 


In [47]:
def compile_lstm(embeddings, shape, settings):
    from keras.utils import multi_gpu_model
    model = Sequential()
    model.add(
        Embedding(
            embeddings.shape[0],
            embeddings.shape[1],
            input_length=shape['max_length'],
            trainable=False,
            weights=[embeddings],
            mask_zero=True
        )
    )
    #the same dense layer is first applied extract the most useful info from embedding layers
    #use_bias=False
    model.add(TimeDistributed(Dense(shape['nr_hidden'], )))
    model.add(Bidirectional(LSTM(shape['nr_hidden'],
                                 recurrent_dropout=settings['dropout'],
                                 #dropout=settings['dropout']
                                     )))
    model.add(Dense(shape['nr_class'], activation='sigmoid'))
    
    model=multi_gpu_model(model,gpus=4)
    model.compile(optimizer=Adam(lr=settings['lr']), loss='categorical_crossentropy',
		  metrics=['accuracy'])
    return model
model = compile_lstm(embeddings, lstm_shape, lstm_settings)

In [48]:

# Replicates `model` on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
#parallel_model = multi_gpu_model(model, gpus=4)

In [50]:
lstm=model.fit(train_X, train_labels, validation_data=(dev_X, dev_labels),
          nb_epoch=nb_epoch, batch_size=batch_size)
model.save('./model/lstm.h5')

/cellar/users/btsui/anaconda3/envs/deep_nlp_/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


AttributeError: 'Tensor' object has no attribute 'assign'

In [ ]:
from tensorflow import python

In [51]:
import tensorflow as tf

In [53]:
#tf.__version__

In [ ]:
!ls /cellar/users/btsui/anaconda3/lib/python3.6/site-packages/tensorflow/python/

In [ ]:
#tf.python
tensorflow/tensorflow/python/client/device_lib

In [ ]:
#from tensorflow import python
#python

In [ ]:
#nvidia-smi
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


In [ ]:
get_available_gpus()

In [ ]:
import tensorflow as tf


In [ ]:
#loss_history = lstm.history['loss']
#numpy_loss_history = numpy.array(loss_history)
#numpy.savetxt("loss_history.txt", numpy_loss_history, delimiter=",")


In [ ]:
#!cat loss_history.txt

In [ ]:
numpy.save('./model/classes.npy', le.classes_)


In [ ]:
val_docs = list(nlp.pipe(dev_texts))
val_X=get_features(val_docs,lstm_shape['max_length'])
predictM=lstm.model.predict_proba(val_X)

In [ ]:
probaDf_multI=pd.DataFrame(data=predictM,columns=le.classes_)

In [ ]:
probaDf_multI.index=pd.MultiIndex.from_arrays([test_df.attrib.values,dev_texts],names=['entity','text'])

In [ ]:
probaDf=probaDf_multI.reset_index()

In [ ]:
predicted_types=probaDf.loc[:,probaDf.columns.isin(myAttribs)].idxmax(axis=1)
print ((probaDf.entity==predicted_types).mean()

### show contingency tables

In [ ]:
minProbThreshold=0.2
confidence_mask=probaDf_multI.max(axis=1)>=minProbThreshold
tmpDf3=probaDf_multI[confidence_mask].idxmax(axis=1).reset_index(name='predicted')

In [ ]:
continDf=tmpDf3.groupby(['predicted','entity']).size().unstack().fillna(0)

In [ ]:
#probaDf_multI

In [ ]:
fig,ax=plt.subplots(figsize=(12,5))
#,cmap="Greens"
continDf=continDf.loc[attribVC_train.index,attribVC_train.index]
sns.heatmap(ax=ax,data=continDf/continDf.sum(axis=0),annot=True,cbar_kws={'label':'% of samples'},center=0.3)


#### Performance of model in validation cohort

In [ ]:
fig,ax=plt.subplots()
for myClass in le.classes_: 
    fpr,tpr,_=metrics.roc_curve((probaDf.entity==myClass),probaDf[myClass])    
    ax.plot(fpr,tpr,label="{myClass}  (AUC: {AUC})".format(myClass=myClass,AUC=str(metrics.auc(fpr,tpr))[:5]))
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.legend()

# scratch

In [ ]:
asdadasd

### for a sentence split

In [ ]:
import nltk
from nltk.util import ngrams

sent="T cell is treated with LPS grow faster"
s=sent.split(' ')
n_gram=2
grams=list(map(lambda L:" ".join(L),list(ngrams(s,n_gram))))
print (grams)
val_docs = list(nlp.pipe(grams))
val_X=get_features(val_docs,lstm_shape['max_length'])
tmpDf=pd.DataFrame(data=lstm.model.predict_proba(val_X),columns=le.classes_,index=grams)
ax=sns.heatmap(tmpDf,cbar_kws={'label': 'Emitted probability'},annot=True)
ax.set_ylabel('')
ax.set_title('{} grams'.format(n_gram))

In [ ]:
val_X=get_features(val_docs,lstm_shape['max_length'])

In [ ]:
!echo $PWD/Data/DEEP_NLP/NLP_spacy/keras_on_sra_data_v2.ipynb

In [ ]:


tmpS3=srsS[srsS.index.get_level_values(1)=='cur_land_use']

In [ ]:
len(tmpS3.value_counts())

In [ ]:
#no data: biomass,chem_mutagen
srsS.head()